In [ ]:
import sys
sys.path.append('../')
import torch
import torchvision.transforms as transforms
from PIL import Image
from utils import get_person_rect, SquarePad, test_transform
from torchvision.models.detection import retinanet_resnet50_fpn_v2, RetinaNet_ResNet50_FPN_V2_Weights

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
person_index = RetinaNet_ResNet50_FPN_V2_Weights.COCO_V1.meta["categories"].count('person')
device = "cpu"
person_model = retinanet_resnet50_fpn_v2(weights=RetinaNet_ResNet50_FPN_V2_Weights.COCO_V1).to(device).eval()
model = torch.load('../m-EfficientNetV2-S.pth').eval().to(device)
img_transform = transforms.Compose([
	SquarePad(),
	transforms.Resize((256, 256)),
])
person_transform = RetinaNet_ResNet50_FPN_V2_Weights.COCO_V1.transforms()

In [ ]:
img = img_transform(
    Image.open('C:/Users/yrasi/Downloads/test/6.jpg')
    .convert('RGB')
)
person_input = person_transform(img).to(device)
person_input = person_input.reshape([1] + list(person_input.shape))

with torch.no_grad():
    boxes = person_model(person_input)
    for box in boxes:
        box['boxes'] = box['boxes'].to(torch.int64)
    img_input = test_transform(
        img.crop(
            get_person_rect(
                boxes[0],
                person_index
            )
        )
    ).to(device)
    img_input = img_input.reshape([1] + list(img_input.shape))
    result = model(img_input)

result = (result.flatten() * 10).round(decimals=1).clip(0, 10)
result = round(float(result.item()), 1)
print(result)
transforms.functional.to_pil_image(img_input[0])